# 閒聊準備工作

In [2]:
# # -*- coding: utf-8 -*-
# import sys
# import os
# reload(sys)
# sys.setdefaultencoding('utf-8')

# 斷詞

In [1]:
import jieba
import jieba.posseg as pseg
import pandas as pd

class JiebaSegmentor:

    def __init__(self, dict_path, userdict=[]):
        self.__dict_path = dict_path
        self.__userdict = userdict
        self.dictionary_init()

    def dictionary_init(self):
        jieba.set_dictionary(self.__dict_path)
        for path in self.__userdict:
            print path
            jieba.load_userdict(path)

    def taiwan_country(self):
        return [u'臺北', u'台北', u'基隆', u'臺中', u'台中', u'臺南', u'台南', u'高雄',
                u'宜蘭', u'桃園', u'新竹', u'苗栗', u'彰化', u'南投', u'嘉義', u'雲林',
                u'屏東', u'臺東', u'台東', u'花蓮', u'澎湖']

    def wordToNumber(self, input_text):

        target = u''
        for s in input_text:

            if (s == u'零') or (s == '0'):
                to_word = u'0'
            elif (s == u'一') or (s == u'壹') or (s == '1'):
                to_word = u'1'
            elif (s == u'二') or (s == u'兩') or (s == u'貳') or (s == '2'):
                to_word = u'2'
            elif (s == u'三') or (s == u'參') or (s == '3'):
                to_word = u'3'
            elif (s == u'四') or (s == u'肆') or (s == '4'):
                to_word = u'4'
            elif (s == u'五') or (s == u'伍') or (s == '5'):
                to_word = u'5'
            elif (s == u'六') or (s == u'陸') or (s == '6'):
                to_word = u'6'
            elif (s == u'七') or (s == u'柒') or (s == '7'):
                to_word = u'7'
            elif (s == u'八') or (s == u'捌') or (s == '8'):
                to_word = u'8'
            elif (s == u'九') or (s == u'玖') or (s == '9'):
                to_word = u'9'
            else:
                to_word = s

        target = target + to_word
        return target

    def input_text_preprocessing(self, input_text):

        if type(input_text) is not unicode:
            input_text = input_text.decode('utf-8')

        # input_text = self.wordToNumber(input_text)
        return input_text

    def get_names(self, input_text):

        input_text = self.input_text_preprocessing(input_text)
        names = []
        words = pseg.cut(input_text)
        print words
        for w, f in words:
            if f.lower() == 'nr':
                names.append(w)
        for name in names:
            print name.encode('utf-8')
        return names

    def lcut(self, input_text):

        input_text = self.input_text_preprocessing(input_text)
        cut_raw = jieba.lcut(input_text)
        key = []

        for k in cut_raw:
            key.append(k)
        df = pd.DataFrame({"word": key})
        return df

    def pseg_lcut(self, input_text):

        input_text = self.input_text_preprocessing(input_text)
        cut_raw = pseg.lcut(input_text)
        key = []
        value = []

        for k, v in cut_raw:
            tag = v
            if k in self.taiwan_country():
                tag = u'ns'
            if len(k) > 1 and tag == u'x':
                tag = u'n'
            key.append(k)
            value.append(tag)
        df = pd.DataFrame({"word": key, "tag": value})
        return df


In [4]:
jieba_dict_path1 = "/home/charles/dataset/jieba/dataset_02/dict_taiwan.txt"
jieba_dict_path2 = "/home/charles/dataset/jieba/dataset_02/userdict.txt"
jieba_dict_path3 = "/home/charles/dataset/jieba/dataset_02/dict.txt.big"
jieba_dict_path4 = "/home/charles/dataset/jieba/dataset_02/dict.txt.small"
segmentor = JiebaSegmentor(jieba_dict_path1, [jieba_dict_path2,jieba_dict_path3,jieba_dict_path4])
df = segmentor.pseg_lcut('請問今天下雨嗎請問今天下雨嗎')
df

Building prefix dict from /home/charles/dataset/jieba/dataset_02/dict_taiwan.txt ...
DEBUG:jieba:Building prefix dict from /home/charles/dataset/jieba/dataset_02/dict_taiwan.txt ...
Loading model from cache /tmp/jieba.uc7e0f43f50d825489eed13ef9e4d17b7.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.uc7e0f43f50d825489eed13ef9e4d17b7.cache
Loading model cost 0.487 seconds.
DEBUG:jieba:Loading model cost 0.487 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


/home/charles/dataset/jieba/dataset_02/userdict.txt
/home/charles/dataset/jieba/dataset_02/dict.txt.big
/home/charles/dataset/jieba/dataset_02/dict.txt.small


,tag,word
0,v,請問
1,t,今天
2,v,下雨
3,y,嗎
4,v,請問
5,t,今天
6,v,下雨
7,y,嗎


# TFIDF

In [24]:
from gensim.corpora import *
from gensim.models import TfidfModel
from datetime import datetime
from gensim import models
from gensim.corpora import Dictionary

class TfidfGensim:

    def __init__(self, model_path, dictionary_path, stopword_path=None,
                 jieba_dict_path=None, jieba_user_dict_path_list=None, segmentor=None):
        self.__model_path = model_path
        self.model = None
        self.model_init()
        self.__stopword_path = stopword_path
        self.stopword_set = set()
        self.stopword_init()
        self.__dictionary_path = dictionary_path
        self.dictionary_init()
        self.jieba_dict_path = jieba_dict_path
        self.jieba_user_dict_path_list = jieba_user_dict_path_list
        if segmentor:
            self.segmentor = segmentor
        else:
            self.jieba_init()

    def stopword_init(self):
        self.stopword_set = set()
        if self.__stopword_path:
            with open(self.__stopword_path,'r') as stopwords:
                for sw in stopwords:
                    self.stopword_set.add(sw.strip('\n').decode('utf-8'))
    
    def model_init(self):
        self.model = TfidfModel.load(self.__model_path)

    def dictionary_init(self):
        self.dictionary = Dictionary.load(self.__dictionary_path)
    
    def jieba_init(self):
        if not self.jieba_dict_path:
            self.segmentor = None
            return
            
        if not self.jieba_user_dict_path_list:
            self.jieba_user_dict_path_list = []
        
        self.segmentor = JiebaSegmentor(self.jieba_dict_path, self.jieba_user_dict_path_list)

    def predict(self, input_text, num=10, min_tfidf=0.2):
        
        if type(input_text) is not unicode:
            input_text = input_text.decode('utf-8')
        
        if not self.segmentor:
            return pd.DataFrame({"word":[], "tfidf":[]})
        
        words = self.segmentor.pseg_lcut(input_text)
        subject_dict={}
        for index, row in words.iterrows():
#             print row['word'] + ' (' + row['tag'] + ')'
            subject_dict.update({row['word']:row['tag']})
            
        corpus = []
        for index, row in words.iterrows():
            if row['word'] not in self.stopword_set:
                corpus.append(row['word'])
            
        corpus_key = self.dictionary.doc2bow(corpus)
        corpus_tfidf = self.model[corpus_key]
        corpus_tfidf = sorted(corpus_tfidf, key=lambda item: item[1], reverse=True)
        id2token = dict(zip(self.dictionary.token2id.values(), self.dictionary.token2id.keys()))
        key = []
        value = []
        
        corpus_tfidf_len = len(corpus_tfidf)
        if corpus_tfidf_len >= num:
            corpus_tfidf_len = num
        for i in range(corpus_tfidf_len):
            
            word = id2token[corpus_tfidf[i][0]]
            tfidf = corpus_tfidf[i][1]
            
            if tfidf <= min_tfidf:
                break
            
#             print 'keyword: ' + word + '(' + subject_dict[word] + ') , tfidf: ' + str(tfidf)
            key.append(word)
            value.append(tfidf)
        
        df = pd.DataFrame({"word":key, "tfidf":value})
        return df

In [25]:
# tfidf_path = "/home/charles/dataset/tfidf_model/v2"
# tfidf_model_path = tfidf_path+"/tfidf.model"
# tfidf_dictionary_path = tfidf_path+"/tfidf_corpus_dict"
# stopwords_path = "/home/charles/dataset/jieba/stopwords.txt"

# tg = TfidfGensim(model_path=tfidf_model_path, dictionary_path=tfidf_dictionary_path, 
#                    stopword_path=stopwords_path, segmentor=segmentor)
# tg.predict('請問今天下雨嗎')

,tfidf,word
0,0.694170,請問
1,0.609174,下雨
2,0.383452,今天


# Word to vector

In [67]:
from gensim import models

class WordToVecGensim:

    def __init__(self, model_path):
        self.__model_path = model_path
        self.model_init()

    def model_init(self):
        self.model = models.Word2Vec.load(self.__model_path)

    def predict(self, input_text, num=10, min_confidence=0.2):

        if type(input_text) is not unicode:
            input_text = input_text.decode('utf-8')

#         print 'synonyms : '
        key = []
        value = []
        try:
            res = self.model.wv.most_similar(input_text)
            for index,item in enumerate(res):
                if index == num:
                    break
                if item[1] <= min_confidence:
                    break
                
                key.append(item[0])
                value.append(item[1])
#             result.append((word, tfidf))

        except KeyError as er:
            print 'no synonyms'

        df = pd.DataFrame({"word":key, "conf":value})
        return df

In [28]:
# w2v_model_path = "/home/charles/dataset/word2vec_model/v2/word2vec_tw.model"
# # del w2v
# w2v = WordToVecGensim(w2v_model_path)

In [29]:
# input_text = '關雲長'
# print 'input_text: ' + input_text
# res = w2v.predict(input_text)
# res = list(res)
# for index,item in enumerate(res):
#     if index == 5:
#         break

#     if item[1] < 0.15:
#         break
#     print item[0] + ': ' + str(item[1])

input_text: 關雲長
synonyms : 
c: o
w: o


/home/charles/anaconda2/lib/python2.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# 問句分類

In [30]:
from datetime import datetime
import regex as re
from abc import ABCMeta, abstractmethod
from collections import OrderedDict
from collections import namedtuple


class CheckQuestion:
    __metaclass__ = ABCMeta

    def __init__(self):
        pass

    class Label(object):
        WHY = 'why'
        WHEN = 'when'
        WHERE = 'where'
        WHO = 'who'
        WHAT = 'what'
        HOW = 'how'
        HOW_MANY = 'how_many'
        STATUS = 'status'
        FEEL = 'feel'
        OTHER = 'other'

    def get_v_subject(self):
        return ['v', 'vd', 'vg', 'vi', 'vn', 'vq', 'vt']

    def get_adj_subject(self):
        return ['a', 'ad', 'ag', 'an']

    def get_people_subject(self):
        return ['nr', 'nrfg', 'nrt']

    def get_location_subject(self):
        return ['ns']

    def get_n_subject(self):
        return ['n', 'ng', 'nt', 'nz']

    def get_eng_subject(self):
        return ['eng']

    def __get_unit_word(self):
        return [u'有沒有', u'什麼', u'哪一個', u'那一個', u'哪個', u'哪些', u'那個', u'那些', u'怎麼', u'哪兒', u'那兒', u'怎', u'哪', u'那']

    def get_adj_word_rule(self):
        return u'(鬼|怪|神祕|神奇)?(的)?'

    def get_unit_word_post_rule(self):

        unit_word = self.__get_unit_word()
        unit_word_rule = ''

        for k, v in enumerate(unit_word):
            unit_word_rule = unit_word_rule + v
            if k < len(unit_word) - 1:
                unit_word_rule = unit_word_rule + '|'

        unit_word_rule = '(' + unit_word_rule + ')' + u'(是)?'

        return unit_word_rule

    def get_unit_word_pre_rule(self):

        unit_word = self.__get_unit_word()
        unit_word_rule = ''

        for k, v in enumerate(unit_word):
            unit_word_rule = unit_word_rule + v
            if k < len(unit_word) - 1:
                unit_word_rule = unit_word_rule + '|'

        unit_word_rule = u'(是)?' + '(' + unit_word_rule + ')'

        return unit_word_rule

    def __repr__(self):
        return "{class_name}".format(class_name=self.__class__.__name__)

    @abstractmethod
    def check(self):
        pass

    @abstractmethod
    def label(self):
        pass

    @abstractmethod
    def target(self):
        pass


class CheckWhy(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckWhy, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """s
        詢問原因:
        label: why
        句型：
        1.(為什麼|爲何|爲啥)
        2.通用疑問句+通用形容句+(原因|理由)
        3.(原因|理由) + 通用疑問句+通用形容句
        """

        # 1
        rule = u'(為什麼|爲何|爲啥)'
        pattern = re.compile(rule)
        #         print rule
        match = pattern.search(input_text)
        if match is not None:
            self.label = self.Label.WHY
            return True

        # 2
        rule = self.get_unit_word_post_rule() + self.get_adj_word_rule() + u'(原因|理由)'
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        if match is not None:
            self.label = self.Label.WHY
            return True

        # 3
        rule = u'(原因|理由)' + self.get_unit_word_pre_rule() + self.get_adj_word_rule()
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        if match is not None:
            self.label = self.Label.WHY
            return True

    def label(self):
        return self.label

    def target(self):
        return self.target


class CheckWhen(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckWhen, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問時間:
        label: when
        """

        rule = u'(什麼時間|什麼時候|什麼年份|什麼月份|什麼年|什麼月|什麼日|什麼天|' + \
               u'哪時|那時|何時|哪一年|哪一月|哪一天|哪年|哪月|哪日|哪天|那一年|那一月|那一天|那年|那月|那日|那天)'
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        if match is not None:
            self.label = self.Label.WHEN
            return True
        

    def label(self):
        return self.label

    def target(self):
        return self.target

    
class CheckWhere(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckWhere, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問地點:
        label: where
        句型：
        1.問單一地點
        2.(在何處|在何地|何地|何處|在哪裡|哪裡|在哪兒|哪兒|在哪|在那裡|那裡|在那兒|那兒|在那)
        3.通用疑問句+通用形容句+(地點|地方|國家|省分|城市|城鎮|{詞性為ns}))
        """
        # 1
        cut_df = self.segmentor.pseg_lcut(input_text)
        if len(cut_df) == 1 and cut_df['tag'][0] in self.get_location_subject():
            self.label = self.Label.WHERE
            self.target = cut_df['word'][0]
            return True

        # 2
        rule = u'(在何處|在何地|何地|何處|在哪裡|哪裡|在哪兒|哪兒|在哪|在那裡|那裡|在那兒|那兒|在那)'
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        if match is not None:
            for index, row in cut_df.iterrows():
                w = row['word']
                n = row['tag']
                if n.lower() in self.get_location_subject():
                    self.label = self.Label.WHERE
                    self.target = w
                    return True

            self.label = self.Label.WHERE
            return True

        # 3
        rule = self.get_unit_word_post_rule() + \
               self.get_adj_word_rule() + \
               u'(地點|地方|國家|省分|城市|城鎮)'
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        # print match
        if match is not None:
            self.label = self.Label.WHERE
            return True

    def label(self):
        return self.label

    def target(self):
        return self.target


class CheckWho(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckWho, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問人:
        label: who
        句型：
        1.問單一人名
        2.(誰|哪位|那位)
        3.通用疑問句+通用形容句+(人)
        4.通用疑問句+通用形容句+{詞性為nr}
        5.{詞性為nr}+通用疑問句+通用形容句
        """
        # 1
        cut_df = self.segmentor.pseg_lcut(input_text)
        if len(cut_df) == 1 and cut_df['tag'][0] in self.get_people_subject():
            self.label = self.Label.WHO
            self.target = cut_df['word'][0]
            return True

        # 2
        rule = u'(誰|哪位|那位)'
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        # print match
        if match is not None:
            for index, row in cut_df.iterrows():
                w = row['word']
                n = row['tag']
                #                 print w
                #                 print n
                if n.lower() in self.get_people_subject():
                    self.label = self.Label.WHO
                    self.target = w
                    return True

            self.label = self.Label.WHO
            return True

        # 3
        rule = self.get_unit_word_post_rule() + \
               self.get_adj_word_rule() + \
               u'(人)'
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        # print match
        if match is not None:
            self.label = self.Label.WHO
            return True

    def label(self):
        return self.label

    def target(self):
        return self.target


class CheckWhat(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckWhat, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問一般事務:
        label: what
        句型：
        1.單一個字{詞性為名詞}
        2.(什麼是|哪個是|那個是)+{詞性為名詞}
        3.{詞性為名詞}+(是什麼|是哪個|是那個)

        """
        # 1
        cut_df = self.segmentor.pseg_lcut(input_text)
        if len(cut_df) == 1 and cut_df['tag'][0] in self.get_n_subject():
            self.label = self.Label.WHAT
            self.target = cut_df['word'][0]
            return True

        # 2
        rule = u'(什麼是|哪個是|那個是)(.*)'
        pattern = re.compile(rule)
        match = pattern.search(input_text)

        if match is not None:
            #             print match.group(1)
            #             print match.group(2)

            content = None
            if match.group(1):
                content = match.group(1)

            if match.group(2):
                content = match.group(2)

            if content is not None:
                cut_df = self.segmentor.pseg_lcut(content)
                #                 print 'cut_df'
                #                 print cut_df
                for index, row in cut_df.iterrows():
                    w = row['word']
                    n = row['tag']
                    #                     if n.lower() not in self.get_people_subject() and \
                    #                             n.lower() not in self.get_location_subject():
                    if n.lower() in self.get_n_subject():
                        self.label = self.Label.WHAT
                        self.target = w
                        return True

        # 3
        rule = u'(.*)(是什麼|是哪個|是那個)'
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)

        if match is not None:
            content = None
            if match.group(1):
                content = match.group(1)

            if content is not None:
                cut_df = self.segmentor.pseg_lcut(content)
                for index, row in cut_df.iterrows():
                    w = row['word']
                    n = row['tag']
                    #                     print w
                    #                     print n
                    #                     if n.lower() not in self.get_people_subject() and \
                    #                             n.lower() not in self.get_location_subject():
                    if n.lower() in self.get_n_subject():
                        self.label = self.Label.WHAT
                        self.target = w
                        return True

    def label(self):
        return self.label

    def target(self):
        return self.target


class CheckHow(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckHow, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問動作:
        label: how
        句型：
        1.(怎麼|怎樣|如何|教我|教導我|請問)+動詞
        """

        # 1
        rule = u'(怎麼|怎樣|如何|教我|教導我|學|學會)(.*)'
        pattern = re.compile(rule)
        match = pattern.search(input_text)

        if match is not None:
            #             print match.group(1)
            #             print match.group(2)

            content = match.group(2)
            if content is not None:
                cut_df = self.segmentor.pseg_lcut(content)
                for index, row in cut_df.iterrows():
                    w = row['word']
                    n = row['tag']
                    if n.lower() in self.get_v_subject():
                        self.label = self.Label.HOW
                        self.target = input_text
                        return True
                    break

    def label(self):
        return self.label

    def target(self):
        return self.target

    
class CheckHowMany(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckHowMany, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問數字:
        label: howmany
        句型：
        1.(多少|幾)
        """

        # 1
        rule = u'(多少|幾)'
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        #         print rule
        if match is not None:
            self.label = self.Label.HOW_MANY
            return True

    def label(self):
        return self.label

    def target(self):
        return self.target

class CheckStatus(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckStatus, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問狀態:
        label: what status
        句型：
        1.多 + 形容詞
        """

        # 1
        rule = u'(多)(.*)'
        pattern = re.compile(rule)
        match = pattern.search(input_text)

        if match is not None:
            #             print match.group(1)
            #             print match.group(2)
            content = match.group(2)
            if content is not None:
                cut_df = self.segmentor.pseg_lcut(content)
                for index, row in cut_df.iterrows():
                    w = row['word']
                    n = row['tag']
                    if n.lower() in self.get_adj_subject():
                        self.label = self.Label.STATUS
                        return True
                    break

    def label(self):
        return self.label

    def target(self):
        return self.target

class CheckOther(CheckQuestion):

    def __init__(self):
        super(CheckOther, self).__init__()
        self.label = 'other'
        self.target = ''

    def check(self, input_text):
        return True

    def label(self):
        return self.label

    def target(self):
        return self.target


class QuestionTypeCheck:

    def __init__(self, segmentor, source='user'):
        self.segmentor = segmentor
        self.source = source
        self.check_pipeline = [CheckWhy(self.segmentor),
                               CheckWhen(self.segmentor),
                               CheckWhere(self.segmentor),
                               CheckWho(self.segmentor),
                               CheckHow(self.segmentor),
                               CheckHowMany(self.segmentor),
                               CheckStatus(self.segmentor),
                               CheckWhat(self.segmentor)]
        #                       CheckFeel(self.segmentor)]

    QuestionTypeTuple = namedtuple('QuestionTypeTuple', {
        'label',
        'target',
    })

    @property
    def source(self):
        return self.source

    @source.setter
    def source(self, source):
        self.source = source

    def check_question_type(self, input_text):
        """
        [ Command ] design pattern
        """
        if type(input_text) is not unicode:
            input_text = input_text.decode('utf-8')

        if self.source == 'ptt':
            # ptt text clean
            input_text = input_text.replace(u'Re: ', u'')
            input_text = re.sub(u"\\[.*?]", u"", input_text)

        label = ''
        target = ''

        # 有被標注到就不往下做
        for cmd in self.check_pipeline:
            if cmd.check(input_text):
                label = '#' + cmd.label
                target = cmd.target
                break

        if label == '':
            cmd = CheckOther()
            cmd.check(input_text)
            label = '#' + cmd.label

        qt = self.QuestionTypeTuple(label=label,
                                    target=target)
        print 'label = ' + qt.label
        print 'target = ' + qt.target
        return qt


# OPEN API

In [56]:
import requests
from bs4 import BeautifulSoup
import functools
from functools import wraps
import regex as re

def cache(func):
    memo = {}

    @wraps(func)
    def _wrapper(*args):
        res = memo.get(args, None)
        if res is not None:
            print 'exist'
            return res
        else:
            res = func(*args)
            memo[args] = res
        return res
    return _wrapper

class WikiPedia:
    
    def __init__(self, lang='zh'):
        self.__lang = lang
   
    def set_lang(self, lang):
        self.__lang = lang

    def wiki_query_list(self, query, lang):
        """
        查詢wiki單字解釋與相關詞API
        """

        url = ("https://"+lang+".wikipedia.org/w/api.php?action=opensearch&search="+query+"&utf8")    
        response = requests.post(url)
        if response.status_code == requests.codes.ok:
            result = response.json()
        #     print result[0]
            if len(result[2]) == 0:
                return None

            content = result[2][0]
            if content == u"" or u"重定向" in content:  
                print 'redirect'
                return None

            return content
        
    def wiki_query_snippet(self, query, lang):
    
        def query_api(query_str, lang):
            
            search_list = []
            url = ("https://"+lang+".wikipedia.org/w/api.php?action=query&list=search&srsearch="+
                    query_str+"&format=json&formatversion=2")    
            response = requests.post(url)
            if response.status_code == requests.codes.ok:

                result = response.json()
                if "query" in result:
                    if "search" in result["query"]:
                        search_list = result["query"]["search"]

            return search_list

        search_list = query_api(query, lang)
        if search_list:

            if search_list[0].has_key("title"):
                title = search_list[0]["title"]
                search_list = query_api(title, lang)  
                if search_list:
                    if search_list[0].has_key("snippet") and\
                        search_list[0]["snippet"]:

                        snippet = search_list[0]["snippet"]
                        snippet = snippet.split(u'。')[0]
                        snippet = re.sub(r'</?\w+[^>]*>','',snippet)+u'。'
                        return snippet
        return None
    
    @cache
    def summery(self, query):
        """
        查詢wiki API
        1.查詢wiki單字解釋與相關詞API
        2.查詢wiki單字解釋片段API
        都以中文wiki百科查詢, 當出現重定向(縮寫或只有簡體文章時發生), 改由查詢wiki單字解釋片段API
        wiki單字解釋片段API的答案不完整, 所以不考慮一開始就呼叫
        爲了提高英文單字準確性, 會使用wiki單字解釋片段API查出的title再當單字查詢一次
        """

        def check_contain_chinese(query):

            for ch in query:
                if u'\u4e00' <= ch <= u'\u9fff':
                    return True
            return False

        if not query:
            return None

        if type(query) is not unicode:
            query = query.decode('utf-8')

    #     if check_contain_chinese(query):
    #         lang = 'zh'
    #     else:
    #         lang = 'en'

        lang = 'zh'

        print 'wiki query : ' + query
        for result in [self.wiki_query_list(query, lang), 
                       self.wiki_query_snippet(query, lang)]:
            if result:
                return result
        return None

class GoogleSearch:
    
    def __init__(self, lang='lang_zh-TW'):
        self.lang = lang
        self.url = 'https://www.google.com.tw/search'
    
    def url(self):
        return self.url
    
    def lang(self):
        return self.lang

    def search(self, query):
        if type(query) is not unicode:
            query = query.decode('utf-8')

        # 查詢參數
        my_params = {'q': query, 'lr':self.lang}

        # 下載 Google 搜尋結果
        r = requests.get(self.url, params = my_params)

        # 確認是否下載成功
        if r.status_code == requests.codes.ok:
          # 以 BeautifulSoup 解析 HTML 原始碼
          soup = BeautifulSoup(r.text, 'html.parser')

          # 觀察 HTML 原始碼
          # print(soup.prettify())

          # 以 CSS 的選擇器來抓取 Google 的搜尋結果
          items = soup.select('div.g > h3.r > a[href^="/url"]')
          for i in items[:5]:
            # 標題
            print(u"標題：" + i.text)
            # 網址
            print(u"網址：" + i.get('href').lstrip('/url?q='))

# 閒聊

In [43]:
jieba_dict_path1 = "/home/charles/dataset/jieba/dict_taiwan.txt"
jieba_dict_path2 = "/home/charles/dataset/jieba/userdict.txt"
jieba_dict_path3 = "/home/charles/dataset/jieba/dict.txt.big"
jieba_dict_path4 = "/home/charles/dataset/jieba/dict.txt.small"
del segmentor
segmentor = JiebaSegmentor(jieba_dict_path1, [jieba_dict_path2,jieba_dict_path3,jieba_dict_path4])

Building prefix dict from /home/charles/dataset/jieba/dict_taiwan.txt ...
DEBUG:jieba:Building prefix dict from /home/charles/dataset/jieba/dict_taiwan.txt ...
Loading model from cache /tmp/jieba.u48306fa201322dcccc3d0c62898fbadc.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.u48306fa201322dcccc3d0c62898fbadc.cache
Loading model cost 0.833 seconds.
DEBUG:jieba:Loading model cost 0.833 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


/home/charles/dataset/jieba/userdict.txt
/home/charles/dataset/jieba/dict.txt.big
/home/charles/dataset/jieba/dict.txt.small


In [44]:
tfidf_path = "/home/charles/dataset/tfidf_model/v2"
tfidf_model_path = tfidf_path+"/tfidf.model"
tfidf_dictionary_path = tfidf_path+"/tfidf_corpus_dict"
stopwords_path = "/home/charles/dataset/jieba/stopwords.txt"

del tg
tg = TfidfGensim(model_path=tfidf_model_path, dictionary_path=tfidf_dictionary_path, 
                   stopword_path=stopwords_path, segmentor=segmentor)

In [68]:
w2v_model_path = "/home/charles/dataset/word2vec_model/v2/word2vec_tw.model"
del w2v
w2v = WordToVecGensim(w2v_model_path)

In [ ]:
qt = QuestionTypeCheck(segmentor)

In [ ]:
wp = WikiPedia()

In [48]:
ww = '臺中今天天氣好晴朗,路邊野花香'
df = segmentor.pseg_lcut(ww)
df
# for index, row in df.iterrows():
#     print row['word']+ '(' + row['tag']+ ')'

,tag,word
0,ns,臺中
1,i,今天天氣
2,a,好
3,a,晴朗
4,x,","
5,s,路邊
6,n,野花
7,ns,香


In [37]:
ww = u'今天天氣好晴朗,今天天氣好晴朗,路邊野花香,晴朗,晴朗,晴朗'
df = tg.predict(ww, num=8, min_tfidf=0.15)
df
# for item in res:
#     print item[0] + '(' + str(item[1]) + ')'

,tfidf,word
0,0.817155,晴朗
1,0.506558,今天天氣
2,0.169780,野花


In [71]:
df = w2v.predict(u'美女', num=5, min_confidence=0.5)
df
# for item in res:
#     print item[0] + '(' + str(item[1]) + ')'

,conf,word
0,0.627519,女明星
1,0.615591,帥哥
2,0.594882,美腿
3,0.592432,童顏
4,0.587876,美胸


In [75]:
def chat(input_text):
    
    # 問句分類
    question_type = qt.check_question_type(input_text)
    
    if question_type.label in ['what', 'where', 'who']:
        wp = WikiPedia()
        print wp.summery(qt.target)
        return
    elif question_type.label in ['how']:
        # google整句丟入
        google_search(qt.target)
        return
        
    # 斷詞
    cut_df = segmentor.pseg_lcut(input_text)
    tag_dict = {}
    for index, row in cut_df.iterrows():
        tag_dict.update({row['word']:row['tag']})

    # 找關鍵字
    keyword_df = tg.predict(input_text, min_tfidf=0.15)
    
    # TFIDF 轉成百分比
    
    keyword_list=[]
    synonyms_list=[]
    
    def filter_by_tag(word):
        v_tag = ['v', 'vd', 'vg', 'vi', 'vn', 'vq', 'vt']
        n_tag = ['n', 'ng', 'nr', 'nrfg', 'nrt', 'ns', 'nt','nz']
        tag = segmentor.pseg_lcut(word)
#         print tag['word'][0] + '(' + tag['tag'][0] + ')'
        if tag['tag'][0] in v_tag or tag['tag'][0] in n_tag:
            return True
        return False

    for index, row in keyword_df.iterrows():

        if filter_by_tag(row['word']):
            keyword_list.append(row['word'])
        
#             w2v_df = w2v.predict(row['word'], num=5, min_confidence=0.5)
#             for index, row in w2v_df.iterrows():
#                 if filter_by_tag(row['word']):
#                     synonyms_list.append(row['word'])
        
    print 'keyword:'
    for i,k in enumerate(keyword_list):
        
        
        print str(i+1) + '. ' + k

#     print 'synonyms ： '
#     for s in synonyms_list:
#         print s
        

In [76]:
chat('台北大屯山托嬰中心在哪裡')

label = #where
target = 台北
keyword:
1. 大屯山
2. 台北
3. 中心


In [77]:
chat('落建洗髮乳有效果嗎')

label = #other
target = 
keyword:
1. 落建
2. 洗髮乳
3. 效果
